This Jupyter notebook is designed to practice the basic functions of SQLAlchemy Core using the `Metaverse Financial Transactions dataset` from *Kaggle*. SQLAlchemy Core offers a direct approach to working with SQL expressions and database structures, providing fine-grained control over database queries and operations without the abstraction layer of SQLAlchemy ORM.

The core focus of this notebook is to demonstrate how to convert a CSV file into a database and leverage SQLAlchemy Core functions to perform various database operations and analyses.

You can access the dataset [here](https://www.kaggle.com/datasets/faizaniftikharjanjua/metaverse-financial-transactions-dataset)



# Packages

In [161]:
from sqlalchemy import create_engine, MetaData, Table, select, func, distinct, case, cast, Float
import pandas as pd

In [2]:
 # function that imports csv file and converts it sqlalchemy database 
def csv_to_db(csv_path, db_name, table_name):
    """
    Load a CSV file using pandas and convert it to a SQLAlchemy database.

    Parameters:
    csv_path (str): The path to the CSV file.
    db_name (str): The name of the database file.
    table_name (str): The name of the SQL table to create or replace.
    """

    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_path)

    # Create a SQLAlchemy engine for the SQLite database
    engine = create_engine(f"sqlite:///{db_name}")

    # Convert the DataFrame to a SQL table
    df.to_sql(table_name, engine, if_exists="replace", index=False)
    

In [3]:
# create database from csv file 
csv_to_db("metaverse_transactions_dataset.csv", "transaction.db", "transaction")

In [4]:
def meta_data(table_name, db_name): 
    """
    Print and explore the metadata of a specified table in the database.
    
    Parameters: 
    table_name (str): The name of the SQL table to retrieve metadata for.
    db_name (str): The name of the SQLite database.
    """
    # Create a SQLAlchemy engine for the SQLite database
    engine = create_engine(f"sqlite:///{db_name}")
    
    # Initialize the metadata object
    metadata = MetaData() 
    
    # Reflect the table from the database via the engine
    db_base = Table(table_name, metadata, autoload_with=engine)
    
    # Print the column name
    print("Table column names: ", db_base.columns.keys())

    # Print full metadata of the table
    print("Metadata of the table:", repr(metadata.tables[table_name]))


In [5]:
meta_data("transaction", "transaction.db")

Table column names:  ['timestamp', 'hour_of_day', 'sending_address', 'receiving_address', 'amount', 'transaction_type', 'location_region', 'ip_prefix', 'login_frequency', 'session_duration', 'purchase_pattern', 'age_group', 'risk_score', 'anomaly']
Metadata of the table: Table('transaction', MetaData(), Column('timestamp', TEXT(), table=<transaction>), Column('hour_of_day', BIGINT(), table=<transaction>), Column('sending_address', TEXT(), table=<transaction>), Column('receiving_address', TEXT(), table=<transaction>), Column('amount', FLOAT(), table=<transaction>), Column('transaction_type', TEXT(), table=<transaction>), Column('location_region', TEXT(), table=<transaction>), Column('ip_prefix', FLOAT(), table=<transaction>), Column('login_frequency', BIGINT(), table=<transaction>), Column('session_duration', BIGINT(), table=<transaction>), Column('purchase_pattern', TEXT(), table=<transaction>), Column('age_group', TEXT(), table=<transaction>), Column('risk_score', FLOAT(), table=<tran

 # Basic Queries: Data retrieval and engine creation

In [19]:
engine = create_engine("sqlite:///transaction.db")
connection = engine.connect()
metadata = MetaData()
transaction = Table("transaction", metadata, autoload_with = engine)
stmt = select(transaction)

In [10]:
# Limit the results to the first rows
stmt_l = stmt.limit(10)
for result in (connection.execute(stmt_l).fetchall()):
    print(result)

('2022-04-11 12:47:27', 12, '0x9d32d0bf2c00f41ce7ca01b66e174cc4dcb0c1da', '0x39f82e1c09bc6d7baccc1e79e5621ff812f50572', 796.9492059264745, 'transfer', 'Europe', 192.0, 3, 48, 'focused', 'established', 18.75, 'low_risk')
('2022-06-14 19:12:46', 19, '0xd6e251c23cbf52dbd472f079147873e655d8096f', '0x51e8fbe24f124e0e30a614e14401b9bbfed5384c', 0.01, 'purchase', 'South America', 172.0, 5, 61, 'focused', 'established', 25.0, 'low_risk')
('2022-01-18 16:26:59', 16, '0x2e0925b922fed01f6a85d213ae2718f54b8ca305', '0x52c7911879f783d590af45bda0c0ef2b8536706f', 778.197389885983, 'purchase', 'Asia', 192.168, 3, 74, 'focused', 'established', 31.25, 'low_risk')
('2022-06-15 09:20:04', 9, '0x93efefc25fcaf31d7695f28018d7a11ece55457f', '0x8ac3b7bd531b3a833032f07d4e47c7af6ea7bace', 300.8383577068122, 'transfer', 'South America', 172.0, 8, 111, 'high_value', 'veteran', 36.75, 'low_risk')
('2022-02-18 14:35:30', 14, '0xad3b8de45d63f5cce28aef9a82cf30c397c6ceb9', '0x6fdc047c2391615b3facd79b4588c7e9106e49f2', 77

In [11]:
# Count the number of rows/transactions in total 
stmt_c = select(func.count()).select_from(transaction)
print("The number of transactions in the database: ", connection.execute(stmt_c).scalar())

The number of transactions in the database:  78600


In [12]:
# Count number of unique values in the transaction_type column
distinct_stmt = select(func.count(func.distinct(transaction.columns.transaction_type)))
print("The number of unique transaction types: ", connection.execute(distinct_stmt).scalar())                     

The number of unique transaction types:  5


In [13]:
def count_transactions_per_column (table_name, column_name): 
    """
    Calculate the number of transactions per unique value in a specified column.
    
    Parameters:
    table (Table): The SQLAlchemy Table object to query.
    column_name (str): The name of the column to group by.

    Returns:
    List[Tuple]: A list of tuples, each containing a unique column value and the count of transactions.
    """
    
    # count number of transaction per each giving column 
    column = table_name.columns[column_name]
    c = select(column, 
               func.count(column)).group_by(column).order_by(func.count(column).desc())
   
    results = connection.execute(c).fetchall()
    
    return results

    

In [14]:
# Number of transactions per transaction type 

results = count_transactions_per_column(transaction, "transaction_type")
for result in results:
    print(f"Transaction Type: {result[0]}, Count: {result[1]}")


Transaction Type: sale, Count: 25040
Transaction Type: purchase, Count: 24940
Transaction Type: transfer, Count: 22125
Transaction Type: scam, Count: 3949
Transaction Type: phishing, Count: 2546


In [15]:
# Number of transaction for each continent
results = count_transactions_per_column(transaction, "location_region")
for result in results: 
    print(f"Region:{result[0]}, Count:{result[1]}")                                                              

Region:North America, Count:15840
Region:Europe, Count:15807
Region:Asia, Count:15731
Region:South America, Count:15669
Region:Africa, Count:15553


In [16]:
# Number of transactions grouped by purchase pattern 
results = count_transactions_per_column(transaction, "purchase_pattern")

for result in results: 
    print(f"Purchase Pattern: {result[0]}, Count:{result[1]}")

Purchase Pattern: high_value, Count:26422
Purchase Pattern: random, Count:26145
Purchase Pattern: focused, Count:26033


In [17]:
# Number of transactions grouped by age_group
results = count_transactions_per_column(transaction, "age_group")

for result in results: 
    print(f"Age Group: {result[0]}, Count:{result[1]}")

Age Group: veteran, Count:26422
Age Group: new, Count:26145
Age Group: established, Count:26033


In [18]:
# Number of transaction grouped by anomaly
results = count_transactions_per_column(transaction, "anomaly")

for result in results: 
    print(f"Anomaly: {result[0]}, Count:{result[1]}")

Anomaly: low_risk, Count:63494
Anomaly: moderate_risk, Count:8611
Anomaly: high_risk, Count:6495


# Basic queries: Aggregation and Summarization

In [42]:
# The total amount of transactions 
amount_select = select(func.round(func.sum(transaction.columns.amount)))
print("The total amount of transactions: ", connection.execute(amount_select).scalar())

The total amount of transactions:  39502387.0


In [39]:
# The average amount of transactions 
avg_select = select(func.round(func.avg(transaction.columns.amount),2))
print("The average amount of transactions: ", connection.execute(avg_select).scalar())

The average amount of transactions:  502.57


In [46]:
# Maximum value in the amount column
mx_select = select(func.round(func.max(transaction.columns.amount),2))
print("The maximum value of the amount: ", connection.execute(mx_select).scalar())

The maximum value of the amount:  1557.15


In [48]:
# Maximum value in the amount column
mn_select = select(func.round(func.min(transaction.columns.amount),2))
print("The minimum value of the amount: ", connection.execute(mn_select).scalar())

The minimum value of the amount:  0.01


In [66]:
# Average amount per transaction type 
avg_select_tt =select(transaction.columns.transaction_type, func.round(func.avg(transaction.columns.amount),2)
                     ).group_by(transaction.columns.transaction_type)
results = connection.execute(avg_select_tt).fetchall()

for result in connection.execute(avg_select_tt).fetchall(): 
    transaction_type, avg_amount = result
    print(f"Transaction type: {transaction_type}, Average amount: {avg_amount}")

Transaction type: phishing, Average amount: 494.99
Transaction type: purchase, Average amount: 506.16
Transaction type: sale, Average amount: 502.82
Transaction type: scam, Average amount: 495.59
Transaction type: transfer, Average amount: 500.37


In [80]:
# Number of transaction grouped hour of the day
results = count_transactions_per_column(transaction, "hour_of_day")

for result in results: 
    print(f"Hour of the day: {result[0]}, Count:{result[1]}")

Hour of the day: 20, Count:3404
Hour of the day: 17, Count:3377
Hour of the day: 8, Count:3324
Hour of the day: 0, Count:3323
Hour of the day: 22, Count:3318
Hour of the day: 21, Count:3318
Hour of the day: 23, Count:3311
Hour of the day: 13, Count:3299
Hour of the day: 11, Count:3299
Hour of the day: 10, Count:3298
Hour of the day: 5, Count:3283
Hour of the day: 12, Count:3282
Hour of the day: 2, Count:3273
Hour of the day: 7, Count:3258
Hour of the day: 3, Count:3256
Hour of the day: 19, Count:3255
Hour of the day: 18, Count:3241
Hour of the day: 9, Count:3241
Hour of the day: 1, Count:3229
Hour of the day: 6, Count:3219
Hour of the day: 4, Count:3209
Hour of the day: 16, Count:3205
Hour of the day: 14, Count:3199
Hour of the day: 15, Count:3179


In [85]:
# Top 10 sending addresses with the most transactions, sorted in descending order of transaction count
sending_select = select(transaction.columns.sending_address, 
                    func.count(transaction.columns.transaction_type)
                   ).group_by(transaction.columns.sending_address
                   ).order_by(func.count(transaction.columns.transaction_type).desc()).limit(10)

results = connection.execute(sending_select).fetchall()

for result in results:
    print(f"Sending address: {result[0]}, Count:{result[1]}")

Sending address: 0x4bfe6fa35149db081520aea5e503946572718756, Count:232
Sending address: 0xb0b2f6fc707fbb7f9d27a9f4fe0cb0d6b39a0155, Count:204
Sending address: 0xf24dd92ee52bb0de1bab87766a4a3f1c17b2c4d9, Count:203
Sending address: 0xceda6c769350a084f02a6e5f2fadf269ecc0c37d, Count:203
Sending address: 0xccbf5a9926606c77db8391185066a8ee3bc0a2af, Count:203
Sending address: 0x91ee0c3b3f87d070e260f6208c52534d36817dae, Count:203
Sending address: 0x1d132c165feddfa72ab5fca8b7dfa522efb53a56, Count:203
Sending address: 0xe0e56a2db04ea53331dc77010b28bc94280c37e5, Count:174
Sending address: 0xd139ad944b14d015cc91df97b6231b6b236b4d33, Count:174
Sending address: 0xcdcace0693e828baa1857ee5b127419bb5e7fc56, Count:174


In [89]:
# Top 10 receiving addresses with the most transactions, sorted in descending order of transaction count
receiving_select = select(transaction.columns.receiving_address, 
                    func.count(transaction.columns.transaction_type)
                   ).group_by(transaction.columns.receiving_address
                   ).order_by(func.count(transaction.columns.transaction_type).desc()).limit(10)

results = connection.execute(receiving_select).fetchall()

for result in results:
    print(f"Sending address: {result[0]}, Count:{result[1]}")

Sending address: 0x86b61ffe9ce4da504e6b7cf3f0925c1941265074, Count:232
Sending address: 0xbc9bc0162fe1ef6624c4f9ea0139576e7407a99d, Count:203
Sending address: 0xadc445c80282504e4af6afeb5681fded2c3d6069, Count:203
Sending address: 0x90c61e65de8543a1e3daeda46d03fea897886666, Count:203
Sending address: 0x897c07cac7f9149d0be00903c55d810f6e42c9cc, Count:203
Sending address: 0x880517c4e61f634eb26e59e702726e2cbae53959, Count:203
Sending address: 0x617b83371b4c65debb6c2e90de89c827ddcb0884, Count:203
Sending address: 0x4e3bf6cc6fa25ce2b65d184fc14d52f375768e3c, Count:203
Sending address: 0x2c3cfb5537f82b3baf41b0869f20b66f47d57c23, Count:203
Sending address: 0x0496b829a3ede37a92aa2b9b101549ed23018cb8, Count:203


In [186]:
# average session duratin for each region
stmt = select(transaction.columns.location_region, 
              func.avg(transaction.columns.session_duration
                      )).group_by(transaction.columns.location_region)

for result in (connection.execute(stmt).fetchall()):
    print(result[0], result[1])

Africa 69.56850768340513
Asia 69.50778717182634
Europe 69.76396533181502
North America 69.35827020202021
South America 70.2272002042249


In [187]:
# average session duratin for each purchase_pattern
stmt = select(transaction.columns.purchase_pattern, 
              func.avg(transaction.columns.session_duration
                      )).group_by(transaction.columns.purchase_pattern)

for result in (connection.execute(stmt).fetchall()):
    print(result[0], result[1])

focused 59.489609341989016
high_value 119.50121111195216
random 29.49152801682922


# Advanced Queries: case and cast

In [160]:
def calculate_percentage(db_name, condition, column_name): 
    """Calculate the percentage of transactions belonging to a specific group (e.g., region, age)
    
    Parameters: 
    db_name (str): Name of the SQLalchemy database
    condition (str): The value/label must match the specified column.
    column_name (str): The column to be evaluated against the condition.

    Return: 
     float: The percentage of transactions belonging to the specified group. 
    
    """
    # Create the case statement 
    case_stmt = case (
        (transaction.columns[column_name] == condition, 1), 
        else_ = 0)

    # Construct the query 
    stmt = select(func.round(func.sum(case_stmt) / cast (func.count(transaction.columns.transaction_type),
                                                         Float ) * 100, 2).label("percent"))
    # Execute the query and return the result
    result = connection.execute(stmt).scalar()
    print(f"The percentage of transaction in {condition}: {result}%")


In [162]:
# for each region calculate the percentage
calculate_percentage(transaction, "Europe", "location_region")
calculate_percentage(transaction, "Asia", "location_region")
calculate_percentage(transaction, "North America", "location_region")
calculate_percentage(transaction, "South America", "location_region")
calculate_percentage(transaction, "Africa", "location_region")

The percentage of transaction in Europe: 20.11%
The percentage of transaction in Asia: 20.01%
The percentage of transaction in North America: 20.15%
The percentage of transaction in South America: 19.94%
The percentage of transaction in Africa: 19.79%


In [170]:
# for each age_group calculate the percentage
calculate_percentage(transaction, "established", "age_group")
calculate_percentage(transaction, "new", "age_group")
calculate_percentage(transaction, "veteran", "age_group")

The percentage of transaction in established: 33.12%
The percentage of transaction in new: 33.26%
The percentage of transaction in veteran: 33.62%


In [172]:
# for each transaction type calculate the percentage 
calculate_percentage(transaction, "phishing", "transaction_type")
calculate_percentage(transaction, "purchase", "transaction_type")
calculate_percentage(transaction, "sale", "transaction_type")
calculate_percentage(transaction, "transfer", "transaction_type")
calculate_percentage(transaction, "scam", "transaction_type")

The percentage of transaction in phishing: 3.24%
The percentage of transaction in purchase: 31.73%
The percentage of transaction in sale: 31.86%
The percentage of transaction in transfer: 28.15%
The percentage of transaction in scam: 5.02%


In [175]:
# for each anomaly calculate the percentage 
calculate_percentage(transaction, "high_risk", "anomaly")
calculate_percentage(transaction, "low_risk", "anomaly")
calculate_percentage(transaction, "moderate_risk", "anomaly")

The percentage of transaction in high_risk: 8.26%
The percentage of transaction in low_risk: 80.78%
The percentage of transaction in moderate_risk: 10.96%


In [178]:
# for each purchase_pattern calculate the percentage 
calculate_percentage(transaction, "focused", "purchase_pattern")
calculate_percentage(transaction, "high_value", "purchase_pattern")
calculate_percentage(transaction, "random", "purchase_pattern")

The percentage of transaction in focused: 33.12%
The percentage of transaction in high_value: 33.62%
The percentage of transaction in random: 33.26%
